# Working With Gradients

There are a couple of ways we can interact with the gradients during and after a backward pass.

In the following example, we save the hidden states of the last layer and do a backward pass on the sum of the logits.

Note two things:

1. We use `inference=False` in the `.forward` call to turn off inference mode. This allows gradients to be calculated. 
2. We can all `.backward()` on a value within the tracing context just like you normally would.

In [3]:
from nnsight import LanguageModel

model = LanguageModel('gpt2', device_map='cuda')

with model.forward(inference=False) as runner:
    with runner.invoke('Hello World') as invoker:

        hidden_states = model.transformer.h[-1].output[0].save()

        logits = model.lm_head.output

        logits.sum().backward()

print(hidden_states.value)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[[ 0.5216, -1.1755, -0.4617,  ..., -1.1919,  0.0204, -2.0075],
         [ 0.9841,  2.2175,  3.5851,  ...,  0.5212, -2.2286,  5.7334]]],
       device='cuda:0', grad_fn=<SliceBackward0>)


If we wanted to see the gradients for the hidden_states, we can call `.retain_grad()` on it and access the `.grad` attribute after execution. 

In [1]:
from nnsight import LanguageModel

model = LanguageModel('gpt2', device_map='cuda')

with model.forward(inference=False) as runner:
    with runner.invoke('Hello World') as invoker:

        hidden_states = model.transformer.h[-1].output[0].save()
        hidden_states.retain_grad()

        logits = model.lm_head.output

        logits.sum().backward()

print(hidden_states.value)
print(hidden_states.value.grad)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[[ 0.5216, -1.1755, -0.4617,  ..., -1.1919,  0.0204, -2.0075],
         [ 0.9841,  2.2175,  3.5851,  ...,  0.5212, -2.2286,  5.7334]]],
       device='cuda:0', grad_fn=<AsStridedBackward0>)
tensor([[[  28.7976, -282.5977,  868.7343,  ...,  120.1742,   52.2264,
           168.6447],
         [  79.4183, -253.6227, 1322.1290,  ...,  208.3981,  -19.5544,
           509.9856]]], device='cuda:0')


Torch also provides hooks into the backward process via the inputs and outputs. NNsight uses these in a similar way as  `.input` and `.output` by also providing `.backward_input` and `.backward_output`.

In [2]:
from nnsight import LanguageModel

model = LanguageModel('gpt2', device_map='cuda')

with model.forward(inference=False) as runner:
    with runner.invoke('Hello World') as invoker:

        hidden_states = model.transformer.h[-1].output[0].save()
        hidden_states_grad = model.transformer.h[-1].backward_output[0].save()
        logits = model.lm_head.output

        logits.sum().backward()

print(hidden_states.value)
print(hidden_states_grad.value)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[[ 0.5216, -1.1755, -0.4617,  ..., -1.1919,  0.0204, -2.0075],
         [ 0.9841,  2.2175,  3.5851,  ...,  0.5212, -2.2286,  5.7334]]],
       device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[[  28.7976, -282.5977,  868.7343,  ...,  120.1742,   52.2264,
           168.6447],
         [  79.4183, -253.6227, 1322.1290,  ...,  208.3981,  -19.5544,
           509.9856]]], device='cuda:0')
